    # ACC: Accuracy	-> (Number of Correct Predictions) / (Total Number of Predictions
    # FPR: False Positive Rate	-> Number of FP / (FP +TN)
    # AUC: Area under the ROC curve	-> scikit-learn
    # Ya: Yield of actives 	-> TP/(TP+FP)
    # EF: Enrichment Factor	-> ((TP)/(TP+FP))((tp+fn)/(tp+tn+fp+fn))
    # REF: Relative Enrichment Factor -> 100*tp/min(tp+fp,tp+fn)

In [96]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve, RocCurveDisplay, auc

In [97]:
# paths
base_dir = "/home/bac/activity_prediction/implementation/"
data_dir = base_dir + "data/results/ACHE/"

In [98]:
def calc_metrics_from_result_df(df: pd.DataFrame,name:str):
    label = df["LABEL"]
    pred = df["PRED"]

    true_positive = 0
    false_positive = 0
    true_negative = 0
    false_negative = 0

    for i, v in enumerate(pred):
        if v == 1 and label[i] == 1:
            true_positive += 1
        elif v == 1 and label[i] == 0:
            false_positive += 1
        elif v == 0 and label[i] == 0:
            true_negative += 1
        else:
            false_negative += 1

    resultdf = pd.DataFrame()
    resultdf["name"] = [name]
    resultdf["ACC"] = [(true_positive + true_negative) / len(label)]
    resultdf["FPR"] = [(false_positive) / (false_positive + true_negative)]
    resultdf["AUC"] = roc_auc_score(label, pred)
    resultdf["EF"] = [
        ((true_positive) / (true_positive + false_positive))
        / ((true_positive + false_negative) / (len(label)))
    ]
    resultdf["REF"] = [
        (100 * true_positive)
        / min((true_positive + false_positive), (true_positive + false_negative))
    ]
    return resultdf


def print_roc_curve(df: pd.DataFrame):
    label = df["LABEL"]
    pred = df["PRED"]

    fpr, tpr, thresholds = roc_curve(label, pred)
    roc_auc = auc(fpr, tpr)
    display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc,                                          estimator_name='example estimator')
    display.plot()  
    plt.show() 

### load results


In [99]:
res = {
    "baseline_rf": pd.read_csv(data_dir + "baseline_rf.csv"),
    "baseline_nn": pd.read_csv(data_dir + "baseline_nn.csv"),
    "baseline_knn": pd.read_csv(data_dir + "baseline_knn.csv"),
    "fe_rf_mdi_nn": pd.read_csv(data_dir + "fe_rf_mdi_nn.csv"),
    "fe_rf_per_nn": pd.read_csv(data_dir + "fe_rf_per_nn.csv"),
    "fe_rf_mdi_knn": pd.read_csv(data_dir + "fe_rf_mdi_knn.csv"),
    "fe_rf_per_knn": pd.read_csv(data_dir + "fe_rf_per_knn.csv"),
    "fe_rf_mdi_rf": pd.read_csv(data_dir + "fe_rf_mdi_rf.csv"),
    "fe_rf_per_rf": pd.read_csv(data_dir + "fe_rf_per_rf.csv"),
}


In [100]:

r = pd.DataFrame()
for k,v in res.items():
    r = pd.concat([r,calc_metrics_from_result_df(v,name=k)])

r

,name,ACC,FPR,AUC,EF,REF
0,baseline_rf,0.807309,0.328467,0.796132,1.413981,92.073171
0,baseline_nn,0.770764,0.299270,0.764999,1.410225,82.926829
0,baseline_knn,0.684385,0.576642,0.662898,1.196626,90.243902
0,fe_rf_mdi_nn,0.750831,0.343066,0.743101,1.363988,82.926829
0,fe_rf_per_nn,0.760797,0.321168,0.754050,1.386721,82.926829
0,fe_rf_mdi_knn,0.551495,0.430657,0.552964,1.098722,59.863946
0,fe_rf_per_knn,0.757475,0.430657,0.741989,1.317248,91.463415
0,fe_rf_mdi_rf,0.534884,0.379562,0.541926,1.089748,59.375000
0,fe_rf_per_rf,0.807309,0.284672,0.799737,1.446348,88.414634
